In [1]:
#CNN으로 모델 구현해보기
import torch
import torch.nn as nn

inputs = torch.Tensor(1,1,28,28)
print('텐서의 크기:{}'.format(inputs.shape))



텐서의 크기:torch.Size([1, 1, 28, 28])


In [2]:
conv1 = nn.Conv2d(1,32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [3]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [5]:
out=conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [6]:
out=pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [7]:
out=conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [8]:
out=pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [9]:
print(out.size(0))
print(out.size(1))
print(out.size(2))
print(out.size(3))

1
64
7
7


In [10]:
out=out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [11]:
fc = nn.Linear(3136, 10)
out=fc(out)
print(out.shape)

torch.Size([1, 10])


In [12]:
#CNN으로 MNIST 분류하기
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)
    
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [13]:
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, 
                          transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train=False, 
                         transform=transforms.ToTensor(), download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                          batch_size=batch_size, shuffle=True, drop_last=True)

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #layer1
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2, stride=2))
        #layer2
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2, stride=2))
        #fc layer
        self.fc = nn.Linear(7*7*64, 10, bias=True)
        
        #fc layer weight initialization
        nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [15]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
total_batch = len(data_loader)
print('총 배치의 수:{}'.format(total_batch))

총 배치의 수:600


In [18]:
for epoch in range(training_epochs+1):
    avg_cost = 0
    for X,Y in data_loader :
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    print('Epoch: {:>4} Cost: {}'.format(epoch+1, avg_cost))

Epoch:    1 Cost: 0.2256559580564499
Epoch:    2 Cost: 0.06301604211330414
Epoch:    3 Cost: 0.04627569019794464
Epoch:    4 Cost: 0.03748538717627525
Epoch:    5 Cost: 0.03145793825387955
Epoch:    6 Cost: 0.02618870697915554
Epoch:    7 Cost: 0.02173391915857792
Epoch:    8 Cost: 0.018384423106908798
Epoch:    9 Cost: 0.01630084402859211
Epoch:   10 Cost: 0.013392644934356213
Epoch:   11 Cost: 0.010059080086648464
Epoch:   12 Cost: 0.009988338686525822
Epoch:   13 Cost: 0.0078106834553182125
Epoch:   14 Cost: 0.0067579192109405994
Epoch:   15 Cost: 0.007783207111060619
Epoch:   16 Cost: 0.005902034696191549


In [20]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:{}%'.format(accuracy.item()*100))

C:\anaconda3\envs\tensorflow\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:98.43999743461609%
